# M2 -Brief 1 - Préoccupation éthique et réglementaire



In [ ]:
import pandas as pd
import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt

## Chargement des données et données brutes

In [ ]:
df = pd.read_csv('data/data-all-68482f115ac04033078508.csv')
df.head()

# Analyse rapide

On commence par virer les éventuels doublons.

In [ ]:
print("Taille avant", len(df))
df = df.drop_duplicates()
print("Taille apres", len(df))

Pas de doublons

---

Infos sur le dataset brute

In [ ]:
df.info()

In [ ]:
df.describe()

Il semble y avoir une valeur incohérente pour le loyer_mensuel qui est negative, on extrait les données avec des loyer négatif pour analyse

In [ ]:
loyer_negatif = df[df['loyer_mensuel'] < 0]
loyer_negatif.describe()

In [ ]:
loyer_negatif

On verra plus tard ce qu'on fait de ces valeurs négatives, suppression des lignes ou les ramené a une valeurs valide.

En première analyse on peux voir que 4 colonnes ont des données manquantes :
* situation_familliale
* historique_credit
* score_credit
* loyer_mensuel

On fait un rendu graphique pour avoir une confirmation visuelle


In [ ]:
msno.matrix(df, 'data', figsize=(10,5))

## Statistique sur les valeurs vides

Pourcentage de valeur vide pour les colonnes


In [ ]:
total_rows = df["montant_pret"].count()
print("Total rows", total_rows)
historique_credits_rows = df["historique_credits"].count()
print("Taille credits:", historique_credits_rows)
situation_familiale_rows = df["situation_familiale"].count()
print("Taille familiale:", situation_familiale_rows)
score_credit_rows = df["score_credit"].count()
print("Taille credit:", score_credit_rows)
loyer_mensuel_rows = df["loyer_mensuel"].count()
print("Taille mensuel:", loyer_mensuel_rows)
historique_credits_pc_vide = 1 - historique_credits_rows/total_rows
print("Pourcent historique_credits:", historique_credits_pc_vide)
situation_familiale_pc_vide = 1 - situation_familiale_rows/total_rows
print("Pourcent situation_familiale:", situation_familiale_pc_vide)
score_credit_pc_vide = 1 - score_credit_rows/total_rows
print("Pourcent score_credit:", score_credit_pc_vide)
loyer_mensuel_pc_vide = 1 - loyer_mensuel_rows/total_rows
print("Pourcent loyer_mensuel:", loyer_mensuel_pc_vide)

print(f"Historique credits: {historique_credits_pc_vide:.4f}")

pc_vide = pd.DataFrame({
    "historique_credits": [historique_credits_pc_vide],
    "situation_familiale": [situation_familiale_pc_vide],
    "score_credit": [score_credit_pc_vide],
    "loyer_mensuel": [loyer_mensuel_pc_vide],
})
print("% de lignes vide")
print(pc_vide)
pc_vide.head()


Utilisation de seaborn pour une analyse plus approfondie

On commence par voir si il y'a une correlation entre les valeurs vide

In [ ]:
msno.heatmap(df, figsize=(8,4))

Il n'y a pas de correlation de présence ou d'absence entre les données vides, elle sont donc indépendante.

## Recherche de données correlées

### Sexe

In [ ]:
# Sexe
sns.pairplot(df, hue="sexe", diag_kind="kde")

### Age

In [ ]:
sns.pairplot(df, hue="age", diag_kind="kde")

Il semble yu avoir une correlation entre l'age et pas mal de valeurs, celles qui peuvent nous interesser sont :
* montant_pret
* loyer_mensuel
* score_credit
* risque_personnel
* historique_credit
* revenu_estime_mois

Même si une correlation semble exister entre l'age et le poids, nous ne nous interesseonrs pas à cette valeur.

### Poids

In [ ]:
sns.pairplot(df, hue="poids")

### Smoker

In [ ]:
# smoker
sns.pairplot(df, hue="smoker", diag_kind="kde")

### sport_licence

In [ ]:
sns.pairplot(df, hue="sport_licence")

### Region

In [ ]:
sns.pairplot(df, hue="region", diag_kind="kde")

## Manipulation des données pour poursuivre l'investigation

### Le cas du niveau d'étude

Les valeurs distincts du niveau d'étude

In [ ]:
df = pd.read_csv('data/data-all-68482f115ac04033078508.csv')
df["niveau_etude"].unique()
df

Mappage avec des valeurs numériques

In [ ]:
df['niveau_etude'] = df['niveau_etude'].map({
    'aucun': 0,
    'bac': 1,
    'bac+2': 2,
    'master': 3,
    'doctorat': 4,
})
df

On va maintenant vérifier les éventuelles correlation.

In [ ]:
sns.pairplot(df, hue="niveau_etude", diag_kind="kde")

## Construction du dataset final

In [ ]:
df = df.drop(axis='columns', columns=['niveau_etude', 'nom', 'prenom', 'sexe', 'taille', 'niveau_etude', 'nationalité_francaise', 'date_creation_compte', ])

## Experimentation libre

In [ ]:
sns.pairplot(df, hue="sexe", diag_kind="kde", vars=["montant_pret", "loyer_mensuel", "poids", "risque_personnel", "score_credit"])

In [ ]:
g = sns.PairGrid(df, hue="sexe", vars=["montant_pret", "loyer_mensuel", "poids", "risque_personnel"])
g.map_diag(sns.histplot)
g.map_offdiag(sns.scatterplot)

In [ ]:
sns.scatterplot(data=df, x="score_credit", y="montant_pret", hue="revenu_estime_mois")

In [ ]:
sns.scatterplot(data=df, x="revenu_estime_mois", y="montant_pret", hue="sexe")

In [ ]:
sns.relplot(data=df, x="sexe", y="montant_pret")

In [ ]:
df.groupby('sexe')["montant_pret"].mean()
df.groupby('sexe')["montant_pret"].size()


In [ ]:
sns.boxplot(y=df['montant_pret'], x=df['sexe'])
plt.show()

In [ ]:
sns.boxplot(x=df['score_credit'], y=df['sexe'])
plt.show()

### Cas du loyer



# Tests


In [ ]:
print(df.shape)
threshold = int(df.shape[1]*0.9)
print(threshold)
df_na = df.dropna(thresh=threshold)
print(df_na['nom'].count())
print(df['nom'].count())
msno.matrix(df_na, 'data', figsize=(10,5))

In [ ]:
!pip install scikit-learn

In [ ]:
from sklearn.impute import KNNImputer


In [ ]:
imputer = KNNImputer(n_neighbors=5)
imputer.fit_transform(df_na)